# M5 Model

This notebook **heavily** references the tutorial: *Speech Command Classification with torchaudio*.
- [Tutorial](https://pytorch.org/tutorials/intermediate/speech_command_classification_with_torchaudio_tutorial.html)

## Derivations
We simplified the tutorial by:
- Wrapping the PyTorch model in PyTorch-Lightning, abstracting away all low-level nitty-gritty loops
- The M5 Model is simplified in code by wrapping common blocks to avoid redundant code: See `cnn_m5.py`
- Using PyTorch-Lightning `Trainer`, we limit the number of batches, to make it run quicker

## M5 Model Reference
[Research Paper](https://arxiv.org/pdf/1610.00087.pdf)
Dai, Wei, et al. "Very deep convolutional neural networks for raw waveforms." 2017 IEEE international conference on acoustics, speech and signal processing (ICASSP). IEEE, 2017.

## Results
Through these settings below, we're able to yield 77% accuracy after 5 epochs, see the TensorBoard for history.
The model has not reach states of overfit, instead, was cut short as it takes a long time.

In [2]:
import pytorch_lightning as pl
import torch

from src.dataset.speech_command_dataset import SpeechCommandDataset
from src.model.cnn_mel2 import CnnMel2
from src.model.lit_wrapper import LitWrapper

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} Backend!")

ds = SpeechCommandDataset(batch_size=64, num_workers=4)  #, dl_kwargs={'pin_memory': True})
train_dl, val_dl, test_dl = ds.dls()

model = LitWrapper(CnnMel2(len(ds.classes), n_freq=32, n_time=100, fs=25), ds.classes, lr=0.0002)

trainer = pl.Trainer(
    default_root_dir="cnn_mel2/",
    max_epochs=15,
    # limit_train_batches=128,
    limit_val_batches=8,
    limit_predict_batches=1,
    # profiler='simple'
    # fast_dev_run=True
)

trainer.fit(model, train_dataloaders=train_dl, val_dataloaders=val_dl)
# pred = trainer.predict(model, dataloaders=test_dl)

Using cuda Backend!


GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.

  | Name  | Type    | Params
----------------------------------
0 | model | CNNMel2 | 82.8 K
----------------------------------
82.8 K    Trainable params
0         Non-trainable params
82.8 K    Total params
0.331     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]